In [1]:
# 특정 개수의 파티션으로 데이터 프레임 만들기
import pandas as pd 
import dask.dataframe as dd

person_IDs = [1,2,3,4,5,6,7,8,9,10]
person_last_names = ['Smith', 'Williams', 'Williams','Jackson','Johnson','Smith','Anderson','Christiansen','Carter','Davidson']
person_first_names = ['John', 'Bill', 'Jane','Cathy','Stuart','James','Felicity','Liam','Nancy','Christina']
person_DOBs = ['1982-10-06', '1990-07-04', '1989-05-06', '1974-01-24', '1995-06-05', '1984-04-16', '1976-09-15', '1992-10-02', '1986-02-05', '1993-08-11']

# 데이터프레임으로 변환 
people_pandas_df = pd.DataFrame({'Person ID': person_IDs, 
              'Last Name': person_last_names, 
              'First Name': person_first_names,
             'Date of Birth': person_DOBs},
            columns=['Person ID', 'Last Name', 'First Name', 'Date of Birth'])

# 판다스를 대스크로 변환
people_dask_df = dd.from_pandas(people_pandas_df, npartitions=2)

In [3]:
print(people_dask_df.divisions) # 파티션 구성의 경계 (0~5(5는 불포함) / (5~9)
print(people_dask_df.npartitions)

(0, 5, 9)
2


In [4]:
# 데이터 프레임에서 행 조사하기
people_dask_df.map_partitions(len).compute()

0    5
1    5
dtype: int64

In [9]:
# 데이터 프레임 재파티셔닝 
people_filtered = people_dask_df[people_dask_df['Last Name'] != 'Williams']
print(people_filtered.map_partitions(len).compute())

people_filtered_reduced = people_filtered.repartition(npartitions=1)
print(people_filtered_reduced.map_partitions(len).compute())

0    3
1    5
dtype: int64
0    8
dtype: int64


In [10]:
print(people_filtered_reduced.repartition(npartitions=2).divisions)

(0, 4, 9)


In [13]:
from dask.diagnostics import ProgressBar

In [25]:
fy14 = dd.read_csv('./data/*2014_.csv', 
                   dtype={'House Number': 'object',
                          'Issuer Command': 'object',
                          'Issuer Squad': 'object',
                          'Time First Observed': 'object',
                          'Unregistered Vehicle?': 'float64',
                          'Violation Description': 'object',
                          'Violation Legal Code': 'object',
                          'Violation Location': 'float64',
                          'Violation Post Code': 'object'})
fy15 = dd.read_csv('./data/*2015.csv')
fy16 = dd.read_csv('./data/*2016.csv')
fy17 = dd.read_csv('./data/*2017.csv', 
                   dtype={'House Number': 'object','Time First Observed': 'object'}
                  )

In [12]:
fy17

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Time First Observed,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Intersecting Street,Date First Observed,Law Section,Sub Division,Violation Legal Code,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
npartitions=32,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,int64,object,object,object,int64,int64,int64,int64,float64,int64,int64,int64,object,object,object,float64,object,object,float64,object,object,int64,int64,object,object,object,object,object,object,float64,int64,object,int64,object,object,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [14]:
fy17.columns

Index(['Summons Number', 'Plate ID', 'Registration State', 'Plate Type',
       'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make',
       'Issuing Agency', 'Street Code1', 'Street Code2', 'Street Code3',
       'Vehicle Expiration Date', 'Violation Location', 'Violation Precinct',
       'Issuer Precinct', 'Issuer Code', 'Issuer Command', 'Issuer Squad',
       'Violation Time', 'Time First Observed', 'Violation County',
       'Violation In Front Of Or Opposite', 'House Number', 'Street Name',
       'Intersecting Street', 'Date First Observed', 'Law Section',
       'Sub Division', 'Violation Legal Code', 'Days Parking In Effect    ',
       'From Hours In Effect', 'To Hours In Effect', 'Vehicle Color',
       'Unregistered Vehicle?', 'Vehicle Year', 'Meter Number',
       'Feet From Curb', 'Violation Post Code', 'Violation Description',
       'No Standing or Stopping Violation', 'Hydrant Violation',
       'Double Parking Violation'],
      dtype='object')

In [15]:
from functools import reduce

columns = [
    set(fy14.columns),
    set(fy15.columns),
    set(fy16.columns),
    set(fy17.columns)
]
common_columns = list(reduce(lambda a, i : a.intersection(i), columns))

In [19]:
common_columns

['Violation Location',
 'Violation County',
 'Violation Description',
 'Violation Legal Code',
 'Issuing Agency',
 'Unregistered Vehicle?',
 'Violation Code',
 'Issuer Command',
 'Vehicle Body Type',
 'Street Code2',
 'Vehicle Make',
 'No Standing or Stopping Violation',
 'Issuer Precinct',
 'Registration State',
 'Days Parking In Effect    ',
 'Hydrant Violation',
 'Feet From Curb',
 'Sub Division',
 'Violation Post Code',
 'Violation Precinct',
 'Street Code1',
 'Summons Number',
 'Violation In Front Of Or Opposite',
 'Plate Type',
 'Meter Number',
 'Time First Observed',
 'Vehicle Color',
 'To Hours In Effect',
 'Plate ID',
 'Vehicle Year',
 'Law Section',
 'Intersecting Street',
 'From Hours In Effect',
 'Issuer Code',
 'Issuer Squad',
 'Street Name',
 'Street Code3',
 'Issue Date',
 'Violation Time',
 'Double Parking Violation',
 'Date First Observed',
 'Vehicle Expiration Date',
 'House Number']

In [23]:
fy17[common_columns].head()

,Violation Location,Violation County,Violation Description,Violation Legal Code,Issuing Agency,Unregistered Vehicle?,Violation Code,Issuer Command,Vehicle Body Type,Street Code2,...,Issuer Code,Issuer Squad,Street Name,Street Code3,Issue Date,Violation Time,Double Parking Violation,Date First Observed,Vehicle Expiration Date,House Number
0,NaN,BX,FAILURE TO STOP AT RED LIGHT,T,V,NaN,7,NaN,SUBN,0,...,0,NaN,ALLERTON AVE (W/B) @,0,07/10/2016,0143A,NaN,0,0,NaN
1,NaN,BX,FAILURE TO STOP AT RED LIGHT,T,V,NaN,7,NaN,SUBN,0,...,0,NaN,ALLERTON AVE (W/B) @,0,07/08/2016,0400P,NaN,0,0,NaN
2,NaN,BX,BUS LANE VIOLATION,T,V,NaN,5,NaN,SUBN,0,...,0,NaN,SB WEBSTER AVE @ E 1,0,08/23/2016,0233P,NaN,0,0,NaN
3,14.0,NY,47-Double PKG-Midtown,NaN,T,NaN,47,T102,REFG,34330,...,359594,J,7th Ave,34350,06/14/2017,1120A,NaN,0,20180630,330
4,13.0,NY,69-Failure to Disp Muni Recpt,NaN,T,NaN,69,T102,DELV,34310,...,364832,M,6th Ave,34330,11/21/2016,0555P,NaN,0,20170228,799


In [26]:
fy14[common_columns].head()

,Violation Location,Violation County,Violation Description,Violation Legal Code,Issuing Agency,Unregistered Vehicle?,Violation Code,Issuer Command,Vehicle Body Type,Street Code2,...,Issuer Code,Issuer Squad,Street Name,Street Code3,Issue Date,Violation Time,Double Parking Violation,Date First Observed,Vehicle Expiration Date,House Number
0,33.0,NaN,NaN,NaN,P,0.0,46,0033,SUBN,13610,...,921043,0000,W 175 ST,21190,08/04/2013,0752A,NaN,0,20140831,712
1,33.0,NY,NaN,NaN,P,0.0,46,0033,VAN,40404,...,921043,0000,W 177 ST,40404,08/04/2013,1240P,NaN,0,20140430,201
2,33.0,NY,NaN,NaN,P,0.0,46,0033,P-U,31190,...,921043,0000,W 163 ST,13610,08/05/2013,1243P,NaN,0,20140228,520
3,33.0,NY,NaN,NaN,P,0.0,46,0033,VAN,11710,...,921043,0000,W 176 ST,12010,08/05/2013,0232P,NaN,0,20141031,517
4,33.0,NY,NaN,NaN,P,0.0,41,0033,TRLR,12010,...,921043,0000,W 174 ST,31190,08/08/2013,1239P,NaN,0,0,590


In [27]:
import numpy as np
import pandas as pd

In [29]:
dtype_tuples = [(x, np.str) for x in common_columns]
dtypes = dict(dtype_tuples)
dtypes

/var/folders/hv/2nnxmnnn0vd4rfr107m9_d1c0000gn/T/ipykernel_5817/2329766034.py:1: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype_tuples = [(x, np.str) for x in common_columns]


{'Violation Location': str,
 'Violation County': str,
 'Violation Description': str,
 'Violation Legal Code': str,
 'Issuing Agency': str,
 'Unregistered Vehicle?': str,
 'Violation Code': str,
 'Issuer Command': str,
 'Vehicle Body Type': str,
 'Street Code2': str,
 'Vehicle Make': str,
 'No Standing or Stopping Violation': str,
 'Issuer Precinct': str,
 'Registration State': str,
 'Days Parking In Effect    ': str,
 'Hydrant Violation': str,
 'Feet From Curb': str,
 'Sub Division': str,
 'Violation Post Code': str,
 'Violation Precinct': str,
 'Street Code1': str,
 'Summons Number': str,
 'Violation In Front Of Or Opposite': str,
 'Plate Type': str,
 'Meter Number': str,
 'Time First Observed': str,
 'Vehicle Color': str,
 'To Hours In Effect': str,
 'Plate ID': str,
 'Vehicle Year': str,
 'Law Section': str,
 'Intersecting Street': str,
 'From Hours In Effect': str,
 'Issuer Code': str,
 'Issuer Squad': str,
 'Street Name': str,
 'Street Code3': str,
 'Issue Date': str,
 'Violation 

In [30]:
fy14 = dd.read_csv('./data/*2014_.csv', dtype=dtypes)

In [31]:
with ProgressBar() : 
    display(fy14[common_columns].head())

[########################################] | 100% Completed |  1.9s


,Violation Location,Violation County,Violation Description,Violation Legal Code,Issuing Agency,Unregistered Vehicle?,Violation Code,Issuer Command,Vehicle Body Type,Street Code2,...,Issuer Code,Issuer Squad,Street Name,Street Code3,Issue Date,Violation Time,Double Parking Violation,Date First Observed,Vehicle Expiration Date,House Number
0,0033,NaN,NaN,NaN,P,0,46,0033,SUBN,13610,...,921043,0000,W 175 ST,21190,08/04/2013,0752A,NaN,0,20140831,712
1,0033,NY,NaN,NaN,P,0,46,0033,VAN,40404,...,921043,0000,W 177 ST,40404,08/04/2013,1240P,NaN,0,20140430,201
2,0033,NY,NaN,NaN,P,0,46,0033,P-U,31190,...,921043,0000,W 163 ST,13610,08/05/2013,1243P,NaN,0,20140228,520
3,0033,NY,NaN,NaN,P,0,46,0033,VAN,11710,...,921043,0000,W 176 ST,12010,08/05/2013,0232P,NaN,0,20141031,517
4,0033,NY,NaN,NaN,P,0,41,0033,TRLR,12010,...,921043,0000,W 174 ST,31190,08/08/2013,1239P,NaN,0,0,590


In [33]:
# Vehicle Year 열 조사하기
with ProgressBar():
    print(fy14['Vehicle Year'].unique().head(10))

[########################################] | 100% Completed | 42.0s
0    2013
1    2012
2       0
3    2010
4    2011
5    2001
6    2005
7    1998
8    1995
9    2003
Name: Vehicle Year, dtype: object


In [34]:
# Vehicle Year 열에서 결측값 확인하기
with ProgressBar() :
    print(fy14['Vehicle Year'].isnull().values.any().compute())

[########################################] | 100% Completed | 41.6s
True


In [37]:
## NYC 주차권 데이터를 위한 최종 스키마
dtypes = {
 'Date First Observed': str,
 'Days Parking In Effect    ': str,
 'Double Parking Violation': str,
 'Feet From Curb': np.float32,
 'From Hours In Effect': str,
 'House Number': str,
 'Hydrant Violation': str,
 'Intersecting Street': str,
 'Issue Date': str,
 'Issuer Code': np.float32,
 'Issuer Command': str,
 'Issuer Precinct': np.float32,
 'Issuer Squad': str,
 'Issuing Agency': str,
 'Law Section': np.float32,
 'Meter Number': str,
 'No Standing or Stopping Violation': str,
 'Plate ID': str,
 'Plate Type': str,
 'Registration State': str,
 'Street Code1': np.uint32,
 'Street Code2': np.uint32,
 'Street Code3': np.uint32,
 'Street Name': str,
 'Sub Division': str,
 'Summons Number': np.uint32,
 'Time First Observed': str,
 'To Hours In Effect': str,
 'Unregistered Vehicle?': str,
 'Vehicle Body Type': str,
 'Vehicle Color': str,
 'Vehicle Expiration Date': str,
 'Vehicle Make': str,
 'Vehicle Year': np.float32,
 'Violation Code': np.uint16,
 'Violation County': str,
 'Violation Description': str,
 'Violation In Front Of Or Opposite': str,
 'Violation Legal Code': str,
 'Violation Location': str,
 'Violation Post Code': str,
 'Violation Precinct': np.float32,
 'Violation Time': str
}

In [39]:
fy14 = dd.read_csv('./data/*2014_.csv', dtype=dtypes, usecols=common_columns)
fy15 = dd.read_csv('./data/*2015.csv', dtype=dtypes, usecols=common_columns)
fy16 = dd.read_csv('./data/*2016.csv', dtype=dtypes, usecols=common_columns)
fy17 = dd.read_csv('./data/*2017.csv', dtype=dtypes, usecols=common_columns)

In [15]:
import pandas as pd 
import dask.dataframe as dd
import pyodbc

In [13]:
username = 'jesse'
password = 'DataScienceRulez'
hostname = 'localhost'
database_name = 'DSAS'
odbc_driver = 'ODBC+Driver+18+for+SQL+Server'

connection_string = 'mssql+pyodbc://{0}:{1}@{2}/{3}?driver={4}'.format(
    username, password, hostname, database_name, odbc_driver
)

In [16]:
data = dd.read_sql_table('violations', connection_string, index_col='Summons Number')

OperationalError: (pyodbc.OperationalError) ('HYT00', '[HYT00] [Microsoft][ODBC Driver 18 for SQL Server]Login timeout expired (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
connection_string